In [20]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
import zipfile

In [21]:
sys.path.append(r'F:\study\ml\python_packages')
import d2l

In [22]:
def load_data_jay_lyrics(): 
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zif:
        with zif.open('jaychou_lyrics.txt') as f:
            corpus_chars =f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    
    return corpus_indices,char_to_idx,idx_to_char,vocab_size

In [23]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

nn.RNN就和nn.linear 一样,固定输入和输出

In [24]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)

h_t = \text{tanh}(w_{ih} x_t + b_{ih} + w_{hh} h_{(t-1)} + b_{hh})

In [25]:
num_steps=35
batch_size=2
state=None
X=torch.rand(num_steps,batch_size,vocab_size)
#Y和X同形状,state_new形状为:num_layers*num_directions,batch,hidden_size
## 参考上面 Y的输出是num_hidden的大小
Y,state_new=rnn_layer(X,state)  
print(Y.shape,len(state_new),state_new[0].shape)
print('X shape : ',X.shape)
print('Y shape : ',Y.shape)
print('state_n shape : ',state_new.shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])
X shape :  torch.Size([35, 2, 1027])
Y shape :  torch.Size([35, 2, 256])
state_n shape :  torch.Size([1, 2, 256])


X(1027) nn.RNN->Y(256) dense(nn.linear)->output(1027)

In [26]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super().__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        print('dense  : ',self.dense)
        self.state=None
    
    def forward(self,inputs,state): #input:(batch,seq_len)
        X=d2l.to_onehot(inputs,self.vocab_size)
        print('X[0] shape : ',X[0].shape)
        
        ## torch.stack 把 list(tensor1,tensor2)变成一个大的tensor
        ## 返回的Y 的特征为是 num_hidden
        
        Y,self.state=self.rnn(torch.stack(X),state)
        print('Y shape : ',Y.shape)
        print('state shape : ',self.state.shape)
        
        ## output 的特征纬度 是vocab_size
        output=self.dense(Y.view(-1,Y.shape[-1]))
        print('output shape : ',output.shape)
        return output,self.state
        

In [27]:
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]
    ## 除了第一个字,剩下都要输出
    for t in range(num_chars+len(prefix)-1):
        X=torch.tensor([output[-1]]).view(1,1)
        if state is not None:
            if isinstance(state,tuple):
                print('state shape2 : ',state.shape)
                print('state[0] : ',state[0])
                print('state[1] : ',state[1])
                state=(state[0],state[1])
            else:
                state=state
        (Y,state)=model(X,state)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [28]:
model=RNNModel(rnn_layer,vocab_size)
predict_rnn_pytorch('分开',10,model,vocab_size,idx_to_char,char_to_idx)

dense  :  Linear(in_features=256, out_features=1027, bias=True)
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  t

'分开飞约飞虽飞虽飞里爽和'

In [29]:
prefix='分开'
output=[char_to_idx[prefix[0]]]
torch.tensor(output[-1]).view(1,1)

tensor([[836]])

In [30]:
d2l.to_onehot(torch.tensor(output[-1]).view(1,1),1027)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.]])]

In [31]:
torch.stack(d2l.to_onehot(torch.tensor(output[-1]).view(1,1),1027)).shape

torch.Size([1, 1, 1027])

In [32]:
X.shape

torch.Size([35, 2, 1027])

In [33]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps):
    corpus_indices = torch.Tensor(corpus_indices)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0:batch_size * batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i:i + num_steps]
        Y = indices[:, i + 1:i + num_steps + 1]
        yield X, Y

In [34]:
data_iter = data_iter_consecutive(corpus_indices, batch_size,num_steps)
for X, Y in data_iter:
    print('X shape : ',X.shape)
    output, state= model(X, state)
    print('output shape: ',output.shape)
    print('state shape: ',state.shape)

X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shap

state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output sha

X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X s

Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] shape :  torch.Size([2, 1027])
Y shape :  torch.Size([35, 2, 256])
state shape :  torch.Size([1, 2, 256])
output shape :  torch.Size([70, 1027])
output shape:  torch.Size([70, 1027])
state shape:  torch.Size([1, 2, 256])
X shape :  torch.Size([2, 35])
X[0] sha

In [35]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size,
                                  corpus_indices, idx_to_char, char_to_idx,
                                  num_epochs, num_steps, lr, clipping_theta,
                                  batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size,
                                          num_steps)
        for X, Y in data_iter:
            if state is not None:
                if isinstance(state, tuple):
                    print('----state is tuple----')
                    state = (state[0].detach(), state[1].detach())
                else:
                    state = state.detach()
            (output, state) = model(X, state)
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())

            optimizer.zero_grad()
            l.backward()
            d2l.grad_clipping(model.parameters(), clipping_theta)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

            try:
                perplexity = math.exp(l_sum / n)
            except OverflowError:
                perplexity = float('inf')
            if (epoch + 1) % pred_period == 0:
                print('epoch %d ,perplexity %f,time %.2f sec' %
                      (epoch + 1, perplexity, time.time() - start))
                for prefix in prefixes:
                    print(
                        ' -',
                        predict_rnn_pytorch(prefix, pred_len, model,
                                            vocab_size, idx_to_char,
                                            char_to_idx))

In [36]:
num_epochs,batch_size,lr,clipping_theta=250,32,1e-3,1e-2
pred_period,pred_len,prefixes=50,50,['分开','不分开']
train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,corpus_indices,
                             idx_to_char,char_to_idx,num_epochs,num_steps,
                              lr,clipping_theta,batch_size,pred_period,
                              pred_len,prefixes)

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 25

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 25

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 25

Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32,

Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1120, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([35, 32, 256])
state shape :  torch.Size([1, 32,

...

output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
sta

Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])


state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
 - 分开 我不多看着 我不要我对你太多 我 一你打我妈妈 就因的话去了口 所有想带你 没有样  能我有话其实
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  t

Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])


X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
ou